In [ ]:
pip install transitions mido python-rtmidi

In [ ]:
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(120)))
track.append(Message('note_on', note=64, velocity=127, time=0))
track.append(Message('note_on', note=32, velocity=127, time=0))
track.append(Message('note_off', note=64, time=480))
track.append(Message('note_off', note=32, time=480))

track.append(Message('note_on', note=64, velocity=127, time=0))
track.append(Message('note_on', note=32, velocity=127, time=0))
track.append(Message('note_off', note=64, time=480))
track.append(Message('note_off', note=32, time=480))
mid.save('new_song.mid')

In [106]:
import random
from transitions import Machine
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

class MidiChord(object):#渡された多重リストのコード進行をmidiで保存する

  ticks_per_beat = 480

  def __init__(self, bpm, beats_per_measure, name):
    """
    コンストラクタ

    Parameters
    ----------
    bpm : int
        譜面のBPM
    beats_per_measure : int
        1小節が何拍子か
    name : string
      保存するファイルの名前(拡張子を除く)
    """

    self.mid = MidiFile( ticks_per_beat=self.ticks_per_beat)
    self.track = MidiTrack()
    self.mid.tracks.append(self.track)
    self.tempo = mido.bpm2tempo(bpm)
    self.beats_per_measure = beats_per_measure

    self.name = name

    self.track.append(MetaMessage('set_tempo', tempo=self.tempo))


  def playChord(self,chord, duration):
    """
    和音を一つ演奏する
    
    Parameters
    ----------
    chord : list
      MIDIのノートナンバーで表現された音高のリスト
    duration : int
      和音の長さ。単位はtick
    """
    
    for tone in chord:
      self.track.append(Message('note_on', note=tone, velocity=127, time=0))
    
    self.track.append(Message('note_off', note=chord[0], velocity=127, time=duration))
    for tone in chord[1:]:
      self.track.append(Message('note_off', note=tone, velocity=127, time=0))

  def playMeasure(self,chord):
    """
    指定された和音で一小節演奏する

    Parameters
    ----------
    chord : list
      MIDIのノートナンバーで表現された音高のリスト
    """
    for i in range(self.beats_per_measure):
      self.playChord(chord, self.ticks_per_beat)

  def playNote(self,chords):
    """
    指定されたコード進行で演奏する

    Parameters
    ----------
    chords : list
      リストで表現された和音のリスト
    """
    for chord in chords:
      self.playMeasure(chord)
    
    self.mid.save(self.name+".mid")
    print("Saved")


class ChordStateMachine(object):
    """
      コード進行を生成するミーリマシン
      トニック、ドミナント、サブドミナントを状態に持ち、各状態に遷移した直後にその上体に対応する和音を生成する

    """
    #状態の定義
    states = ['tonic', 'dominant', 'subdominant', 'end']

    
    majorschale = [0,2,4,5,7,9,11,12,14,16,17,19,21,23,24]
    tonic_root = [1,3,6]#第n音が根音 -1するとmajorschaleのインデックス
    dominant_root = [5,7]
    subdominant_root = [2,4]

    root_dict = {"tonic":tonic_root, "dominant":dominant_root, "subdominant":subdominant_root}

    _past_root = -1


    #初期化（ステートマシンの定義：とりうる状態の定義、初期状態の定義、各種遷移と紐付くアクションの定義）
    def __init__(self, name, initial):
        """
        Parameters
        ----------
        initial: string
          初期状態(statesの要素のどれか)
        """
        self.name = name
        self.machine = Machine(model=self, states=ChordStateMachine.states, initial=initial, auto_transitions=False)
        self.machine.add_transition(trigger="0",      source='tonic',  dest='=', after='soundDst')
        self.machine.add_transition(trigger="1",      source='tonic',  dest='dominant', after='soundDst')
        self.machine.add_transition(trigger="2",      source='tonic',  dest='subdominant', after='soundDst')
        self.machine.add_transition(trigger="0",      source='dominant',  dest='tonic', after='soundDst')
        self.machine.add_transition(trigger='0',      source='subdominant',  dest='tonic', after='soundDst')
        self.machine.add_transition(trigger='1',      source='subdominant',  dest='dominant', after='soundDst')

        self.chords = []

    #以下、遷移時のアクション

    def chooseChord(self):
      """
      現状態に基づいてコードを出力する(根音からの相対値)
      """
      roots = self.root_dict[self.state]
      root = roots[random.randrange(len(roots))]
      while root == self._past_root:
        root = roots[random.randrange(len(roots))]

      self._past_root = root
      triad = [self.majorschale[root -1], self.majorschale[root -1 + 2], self.majorschale[root -1 + 4]]
      return triad


    def soundDst(self):
      """
        状態遷移直後に呼び出され、コード進行に新たなコードを加える
      """
      chord = self.chooseChord()
      print(self.state, chord)
      self.chords.append(chord)


class ChordConvert(object):
  """
    コード進行の調を定める
  """
  def __init__(self, scale_tonic):
    """
      Parameters
      ----------
      scale_tonic : int
        MIDIのノートナンバーで表現された調の主音
    """
    self.scale_tonic = scale_tonic
    self.score = []
  
  def makeChords(self,normalized_chords):
    """
      根音からの相対値で表現されているコード進行を調に合わせる

      Parameters:
      normalized_chords : list
        根音からの相対値で表現されているコード進行
        リストで表現された和音のリストになっている
    """
    chords = [[tone + self.scale_tonic for tone in normalized_chord] for normalized_chord in normalized_chords]

    return chords


chord_generator = ChordStateMachine("chord", "tonic")
chord_converter = ChordConvert(62)
midi_player = MidiChord(bpm=120, beats_per_measure=4, name="testsound")

threshold = 10
endchord = "tonic"

cnt = 0
while(1):
  if(chord_generator.state == "end"):
    break
  triggers = chord_generator.machine.get_triggers(chord_generator.state)

  chord_generator.trigger(triggers[random.randrange(len(triggers))])
  cnt+=1
  if(cnt > threshold and chord_generator.state == endchord):
    break

print(chord_generator.chords)

chords = chord_converter.makeChords(chord_generator.chords)
midi_player.playNote(chords)

  







tonic [9, 12, 16]
subdominant [2, 5, 9]
tonic [9, 12, 16]
subdominant [2, 5, 9]
dominant [11, 14, 17]
tonic [0, 4, 7]
tonic [4, 7, 11]
dominant [7, 11, 14]
tonic [9, 12, 16]
dominant [11, 14, 17]
tonic [9, 12, 16]
[[9, 12, 16], [2, 5, 9], [9, 12, 16], [2, 5, 9], [11, 14, 17], [0, 4, 7], [4, 7, 11], [7, 11, 14], [9, 12, 16], [11, 14, 17], [9, 12, 16]]
Saved


In [ ]:
list = [[1,2,3],[1,2,3],[1,2,3],[1,2,3]]

print(list)

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
